In [7]:
# !gdown 1uNAYBPerV6IaiWXLPF4Efkxb4Gg-U4Cz

In [8]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [9]:
# if IN_COLAB:
#   # !apt-get install openjdk-8-jdk-headless -qq > /dev/null
#   # !wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
#   # !tar xf spark-3.1.1-bin-hadoop3.2.tgz
#   # !pip install -q findspark
#   # import os
#   # os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#   # os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
#     !apt-get install openjdk-8-jdk-headless -qq > /dev/null
#     !wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
#     !tar xf spark-3.3.2-bin-hadoop3.tgz
#     !mv spark-3.3.2-bin-hadoop3 spark
#     !pip install -q findspark
#     import os
#     os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
#     os.environ["SPARK_HOME"] = "/content/spark"

In [10]:
# import findspark
# findspark.init()

In [11]:
spark_url = 'local'

In [13]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master(spark_url)\
        .appName('time-taken-teller')\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

KeyboardInterrupt: 

In [ ]:
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('time-taken-teller')\
        .getOrCreate()

In [ ]:
path = '../data/bangkok_traffy.csv'
df = spark.read.option("delimiter", ",").option("header", True).csv(path)

In [ ]:
df

In [ ]:
# columns_to_drop = ['photo', 'photo_after', 'address', 'star']
columns_to_drop = ['ticket_id', 'organization','comment','photo','photo_after','coords','address','star','count_reopen']
df = df.drop(*columns_to_drop)

In [ ]:
df.show(5)

In [ ]:
from pyspark.sql.functions import avg, min, max, countDistinct, split, explode
from pyspark.sql.functions import col
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import col, udf

In [ ]:
df.show(5), df.count()

In [ ]:
df = df.filter((col('province').rlike('กรุงเทพ|bangkok')) & (col('province').isNotNull()))
df = df.drop(*['province'])

In [ ]:
df.show(5), df.count()

In [ ]:
df = df.dropna(subset=['subdistrict', 'district'])
df.show(5), df.count()

In [ ]:
# Define a user-defined function (UDF) to convert the column values
@udf(ArrayType(StringType()))
def split_and_strip(value):
    return [e.strip() for e in value.strip('{}').split(',') if e]

# Convert 'type' column to string type
df = df.withColumn('type', col('type').cast(StringType()))

# Apply the UDF to split and strip the values in 'type' column
df = df.withColumn('type', split_and_strip(col('type')))

In [ ]:
df.show(5), df.count()

In [ ]:
df = df.filter(col('state').rlike('เสร็จ'))

In [ ]:
from pyspark.sql.functions import col, datediff

# Convert 'timestamp' and 'last_activity' columns to datetime
df = df.withColumn('timestamp', col('timestamp').cast('timestamp'))
df = df.withColumn('last_activity', col('last_activity').cast('timestamp'))

# Calculate the difference in days
df = df.withColumn('time', datediff(col('last_activity'), col('timestamp')))

In [ ]:
df.show(5)

In [ ]:
from pyspark.sql.functions import col, month, year, quarter, dayofweek

# Extract the month
df = df.withColumn('month', month(col('timestamp')))

# Extract the year
df = df.withColumn('year', year(col('timestamp')))

# Extract the quarter of the year
df = df.withColumn('quarter', quarter(col('timestamp')))

# Determine if the day is a weekend
df = df.withColumn('is_weekend', (dayofweek(col('timestamp')) >= 6).cast('integer'))

In [ ]:
df.show(5)

In [ ]:
df.dtypes

In [ ]:
# preprocess_w_spark

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

def array_to_string(my_list):
    return '[' + ','.join([str(elem) for elem in my_list]) + ']'

array_to_string_udf = udf(array_to_string, StringType())

df = df.withColumn('type', array_to_string_udf(df["type"]))

In [ ]:
df.show(5)

In [ ]:
df.columns

In [ ]:
df.na.drop(subset=['type',
 'subdistrict',
 'district',
 'timestamp',
 'state',
 'last_activity',
 'time',
 'month',
 'year',
 'quarter',
 'is_weekend'])

In [ ]:
!export PYSPARK_PYTHON=/usr/bin/python3
export SPARK_DRIVER_PYTHON=/usr/bin/python3

In [ ]:
df.show(5)

In [ ]:
# df = df.dropna(subset=['subdistrict', 'district'])

In [ ]:
# df.write.option("header",True).csv("/tmp/spark_output/datacsv")

In [ ]:
df.coalesce(1).write.option("header",True).csv("dffile.csv")